In [25]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
import numpy as np

from torch import nn
from pytorch_tabnet.tab_model  import TabNetClassifier 
from pytorch_tabnet.metrics import Metric
from sklearn.metrics import f1_score


In [26]:
df = pd.read_csv('./Tabnet_Raw_final.csv')
df

,회사명,거래소코드,회계년도,자기자본구성비율,설비투자효율,총자본투자효율,이자보상배율(이자비용),유동비율,당좌비율,부채비율,...,고정비율,R&D비율,채무부담비율,거래량회전율,로그시가총액,수정거래량,거래량증가율,시가총액증가율,시가총액,부실
0,(주)CMG제약,58820,2011,66.05,89.82,19.56,-9.650000e+00,200.97,165.03,42.33,...,53.829572,0.133525,54.007487,9.350932,11.532230,52414.645542,2.580317,-0.298137,3.405885e+11,0
1,(주)CMG제약,58820,2012,55.32,1749.87,1.66,-1.518000e+01,693.49,649.98,21.50,...,29.462742,0.738537,90.380052,5.391525,11.837788,34146.601343,-0.128726,1.020962,6.883163e+11,0
2,(주)CMG제약,58820,2013,87.63,71.28,34.29,1.920000e+00,694.71,610.74,11.23,...,22.052432,0.659019,84.695602,1.578308,11.993986,12009.124638,-0.615921,0.432840,9.862469e+11,0
3,(주)CMG제약,58820,2014,84.39,79.30,31.04,1.000000e+09,324.58,268.23,17.79,...,38.434202,0.512175,96.059537,1.535762,11.899311,14531.756854,-0.026957,-0.195872,7.930690e+11,0
4,(주)CMG제약,58820,2015,82.80,144.10,15.96,1.000000e+09,296.85,235.21,18.91,...,38.076592,3.076375,88.790984,8.023355,12.127791,46607.904671,4.325087,0.692309,1.342118e+12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10483,휴림로봇(주),90710,2019,66.62,0.00,-14.14,-4.184000e+01,198.14,170.80,46.50,...,28.948701,3.437877,102.076039,14.196914,12.335962,44026.650099,0.281562,-0.295034,2.167517e+12,0
10484,휴젤(주),145020,2016,93.08,21.47,70.84,1.000000e+09,760.09,692.05,7.27,...,21.471858,3.273205,23.935906,3.005742,12.923972,3.861906,10.662331,1.574018,8.394052e+12,0
10485,휴젤(주),145020,2016,93.08,21.47,70.84,1.000000e+09,760.09,692.05,7.27,...,21.471858,3.273205,23.935906,3.005742,12.923972,3.861906,10.662331,1.574018,8.394052e+12,0
10486,흥아해운(주),3280,2018,9.10,201.90,4.52,-1.830000e+00,34.67,28.66,427.21,...,673.576968,0.000000,359.315426,3.706850,12.006935,112546.895592,-0.556525,-0.237193,1.016097e+12,1


In [27]:
df.replace((np.inf, -np.inf), np.nan, inplace=True)

In [28]:
df.dropna(inplace=True)

In [29]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA', 'EBTA',
       'OM', '종업원수증가율', '영업이익변화율', '매출액변화율', '당기순이익변화율', 'DOL', 'DFL',
       'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률', '고정비율', 'R&D비율', '채무부담비율',
       '거래량회전율', '로그시가총액', '수정거래량', '거래량증가율', '시가총액증가율', '시가총액', '부실'],
      dtype='object')

In [16]:
train = df[df['회계년도'] <= 2016]
test = df[df['회계년도'] > 2016]

In [31]:
X_train = train.drop(['부실', '회사명', '회계년도'], axis=1)
X_test = test.drop(['부실', '회사명', '회계년도'], axis=1)
y_train = train['부실']
y_test = test['부실']

In [32]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) 

In [33]:
scaler = StandardScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test) 

In [36]:
scaler = StandardScaler()
scaler.fit(y_train.reshape(-1, 1))
y_train = scaler.transform(y_train.reshape(-1, 1)) 

AttributeError: 'Series' object has no attribute 'reshape'

In [21]:
y_test.shape

(3801,)

In [60]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split


# Preprocess data using pandas
target_col = '부실'
#cat_cols = ['categorical_feature_1', 'categorical_feature_2']
num_cols = ['자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA', 'EBTA',
       'OM', '종업원수증가율', '영업이익변화율', '매출액변화율', '당기순이익변화율', 'DOL', 'DFL',
       'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률', '고정비율', 'R&D비율', '채무부담비율',
       '거래량회전율', '로그시가총액', '수정거래량', '거래량증가율', '시가총액증가율', '시가총액']

#train_cat_feats = train[cat_cols].values
train_num_feats = train[num_cols].values
train_targets = train[target_col].values

#val_cat_feats = test[cat_cols].values
val_num_feats = test[num_cols].values
val_targets = test[target_col].values

# Create TabNet classifier model
n_epochs = 180
batch_size = 90
num_workers = 0
max_epochs_no_improvement = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = TabNetClassifier(
    n_d=15,
    n_a=15,
    n_steps=3,
    gamma=2.3,
    n_independent=2,
    n_shared=2,
    #cat_idxs=[i for i, col in enumerate(cat_cols)],
    #cat_dims=[len(np.unique(df[col])) for col in cat_cols],
    cat_emb_dim=1,
    optimizer_fn=torch.optim.SGD,
    optimizer_params=dict(lr=1e-3),
    scheduler_params=dict(mode="min", patience=max_epochs_no_improvement, min_lr=1e-5, factor=0.5, verbose=True),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    mask_type="sparsemax",
    verbose=20,
    device_name=device
)

# Train TabNet classifier model
model.fit(
    X_train=train_num_feats,
    y_train=train_targets,
    eval_set=[(val_num_feats, val_targets)],
    eval_name=['val'],
    max_epochs=n_epochs,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=True
)


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 1.49789 | val_auc: 0.67486 |  0:00:01s
Epoch    14: reducing learning rate of group 0 to 5.0000e-04.
epoch 20 | loss: 1.294   | val_auc: 0.68363 |  0:00:27s
Epoch    25: reducing learning rate of group 0 to 2.5000e-04.

Early stopping occurred at epoch 31 with best_epoch = 21 and best_val_auc = 0.68841


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [61]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Predict target values for validation set
val_preds = model.predict(val_num_feats)

# Generate confusion matrix
conf_matrix = confusion_matrix(val_targets, val_preds)
print("Confusion Matrix:\n", conf_matrix)

# Generate classification report
class_report = classification_report(val_targets, val_preds)
print("Classification Report:\n", class_report)

# Calculate accuracy score
acc_score = accuracy_score(val_targets, val_preds)
print("Accuracy Score:", acc_score)


Confusion Matrix:
 [[ 521 3140]
 [  19  121]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.14      0.25      3661
           1       0.04      0.86      0.07       140

    accuracy                           0.17      3801
   macro avg       0.50      0.50      0.16      3801
weighted avg       0.93      0.17      0.24      3801

Accuracy Score: 0.16890292028413575


In [62]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predict target values for validation set
val_preds = model.predict(val_num_feats)

# Calculate evaluation metrics
acc = accuracy_score(val_targets, val_preds)
prec = precision_score(val_targets, val_preds)
rec = recall_score(val_targets, val_preds)
f1 = f1_score(val_targets, val_preds)
roc_auc = roc_auc_score(val_targets, val_preds)

# Print evaluation metrics
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)


Accuracy: 0.16890292028413575
Precision: 0.037105182459368294
Recall: 0.8642857142857143
F1 Score: 0.07115554248750368
ROC AUC Score: 0.5032982791586998


In [165]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split


# Preprocess data using pandas
target_col = '부실'
#cat_cols = ['categorical_feature_1', 'categorical_feature_2']
num_cols = ['자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA', 'EBTA',
       'OM', '종업원수증가율', '영업이익변화율', '매출액변화율', '당기순이익변화율', 'DOL', 'DFL',
       'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률', '고정비율', 'R&D비율', '채무부담비율',
       '거래량회전율', '로그시가총액', '수정거래량', '거래량증가율', '시가총액증가율', '시가총액']

#train_cat_feats = train[cat_cols].values
train_num_feats = train[num_cols].values
train_targets = train[target_col].values

#val_cat_feats = test[cat_cols].values
val_num_feats = test[num_cols].values
val_targets = test[target_col].values

# Create TabNet classifier model
n_epochs = 100
batch_size = 90
num_workers = 0
max_epochs_no_improvement = 10
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = TabNetClassifier(
    n_d=10,
    n_a=10,
    n_steps=3,
    gamma=1.3,
    n_independent=2,
    n_shared=2,
    #cat_idxs=[i for i, col in enumerate(cat_cols)],
    #cat_dims=[len(np.unique(df[col])) for col in cat_cols],
    cat_emb_dim=1,
    optimizer_fn=torch.optim.SGD,
    optimizer_params=dict(lr=1e-3),
    scheduler_params=dict(mode="min", patience=max_epochs_no_improvement, min_lr=1e-5, factor=0.5, verbose=True),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    mask_type="sparsemax",
    verbose=10,
    device_name=device
)

# Train TabNet classifier model
model.fit(
    X_train=train_num_feats,
    y_train=train_targets,
    eval_set=[(val_num_feats, val_targets)],
    eval_name=['val'],
    max_epochs=n_epochs,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=True)

c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.58024 | val_auc: 0.49742 |  0:00:01s
epoch 10 | loss: 0.54771 | val_auc: 0.41898 |  0:00:14s

Early stopping occurred at epoch 11 with best_epoch = 1 and best_val_auc = 0.50613


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [328]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split

# Preprocess data using pandas
target_col = '부실'
#cat_cols = ['categorical_feature_1', 'categorical_feature_2']
num_cols = ['자기자본구성비율', '설비투자효율', '총자본투자효율', '이자보상배율(이자비용)',
       '유동비율', '당좌비율', '부채비율', '총자본정상영업이익률', '매출액정상영업이익률', '매출액순이익률',
       '자기자본순이익률', '매출채권회전률', '재고자산회전률', '총자본회전률', '순운전자본비율', '매출액증가율',
       '총자본증가율', '유동자산증가율', '유형자산증가율', '영업이익증가율', '순이익증가율', 'RETA', 'EBTA',
       'OM', '종업원수증가율', '영업이익변화율', '매출액변화율', '당기순이익변화율', 'DOL', 'DFL',
       'EV/EBITDA', '영업활동으로 인한 현금흐름', '금융비용부담률', '고정비율', 'R&D비율', '채무부담비율',
       '거래량회전율', '로그시가총액', '수정거래량', '거래량증가율', '시가총액증가율', '시가총액']

#train_cat_feats = train[cat_cols].values
train_num_feats = train[num_cols].values
train_targets = train[target_col].values

#val_cat_feats = test[cat_cols].values
val_num_feats = test[num_cols].values
val_targets = test[target_col].values

# Create TabNet classifier model
n_epochs = 200
batch_size = 1024
num_workers = 0
max_epochs_no_improvement = 20
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = TabNetClassifier(
    n_d=64,
    n_a=64,
    n_steps=5,
    gamma=1.5,
    n_independent=2,
    n_shared=3,
    #cat_idxs=[i for i, col in enumerate(cat_cols)],
    #cat_dims=[len(np.unique(data[col])) for col in cat_cols],
    cat_emb_dim=1,
    lambda_sparse=0,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    scheduler_params=dict(mode="min", patience=max_epochs_no_improvement, min_lr=1e-5, factor=0.5, verbose=True),
    scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
    mask_type="sparsemax",
    verbose=10,
    device_name=device
)

# Train TabNet classifier model
model.fit(
    X_train=train_num_feats,
    y_train=train_targets,
    eval_set=[(val_num_feats, val_targets)],
    eval_name=['val'],
    max_epochs=n_epochs,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=True
)

c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.51455 | val_auc: 0.46352 |  0:00:02s
epoch 10 | loss: 0.28484 | val_auc: 0.39315 |  0:00:28s

Early stopping occurred at epoch 10 with best_epoch = 0 and best_val_auc = 0.46352


c:\Users\seopa\OneDrive\Documents\6기\.venv\lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [329]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Predict target values for validation set
val_preds = model.predict(val_num_feats)

# Generate confusion matrix
conf_matrix = confusion_matrix(val_targets, val_preds)
print("Confusion Matrix:\n", conf_matrix)

# Generate classification report
class_report = classification_report(val_targets, val_preds)
print("Classification Report:\n", class_report)

# Calculate accuracy score
acc_score = accuracy_score(val_targets, val_preds)
print("Accuracy Score:", acc_score)


Confusion Matrix:
 [[2210  295]
 [  78    9]]
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.88      0.92      2505
           1       0.03      0.10      0.05        87

    accuracy                           0.86      2592
   macro avg       0.50      0.49      0.48      2592
weighted avg       0.93      0.86      0.89      2592

Accuracy Score: 0.8560956790123457


In [330]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Predict target values for validation set
val_preds = model.predict(val_num_feats)

# Calculate evaluation metrics
acc = accuracy_score(val_targets, val_preds)
prec = precision_score(val_targets, val_preds)
rec = recall_score(val_targets, val_preds)
f1 = f1_score(val_targets, val_preds)
roc_auc = roc_auc_score(val_targets, val_preds)

# Print evaluation metrics
print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("ROC AUC Score:", roc_auc)

Accuracy: 0.8560956790123457
Precision: 0.029605263157894735
Recall: 0.10344827586206896
F1 Score: 0.04603580562659847
ROC AUC Score: 0.49284190240209236
